In [1]:
import numpy as np  
import functools
import itertools
import matplotlib.pyplot as plt 
from scipy   import  linalg as LA 
import json

from qiskit import IBMQ, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import ParameterVector

from qiskit.utils                     import QuantumInstance
from qiskit.opflow                    import Z, I, X, Y
from qiskit.opflow         			  import PauliOp, SummedOp

from pauli_function import *
from pVQD			import *
from ansatze        import *

/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_5742/2054964384.py:13: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow                    import Z, I, X, Y


In [2]:
# Initialize system parameters for Ising

spins   = 3
V       = 0.25
g       = 1.0
dt      = 0.05
n_steps = 40

# Algorithm parameters

ths = 0.99999
depth = 3


### Example circ

ex_params = np.zeros((depth+1)*spins +depth*(spins-1))
wfn = hweff_ansatz(spins,depth,ex_params)


### Shift
shift  = np.array(len(ex_params)*[0.01])

print("Initial shift:",shift)


### Generate the Hamiltonian
H = generate_ising(spins,V,g)

print(wfn)
print(H)

### Backend
shots = 8000
backend  = Aer.get_backend('qasm_simulator')
instance = QuantumInstance(backend=backend,shots=shots)

### Prepare the observables to measure
obs = {}
# Magnetization

for i in range(spins):
	obs['Sz_'+str(i)]      = PauliOp(generate_pauli([],[i],spins),1.0) 
	obs['Sx_'+str(i)]      = PauliOp(generate_pauli([i],[],spins),1.0)
	obs['Sy_'+str(i)]      = PauliOp(generate_pauli([i],[i],spins),1.0)


for (name,pauli) in obs.items():
	print(name)
	print(pauli)

Initial shift: [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01]
     ┌───────┐ ░                  ░ ┌───────┐ ░                  ░ ┌───────┐ ░ »
q_0: ┤ Rx(0) ├─░──■───────────────░─┤ Ry(0) ├─░──■───────────────░─┤ Rx(0) ├─░─»
     ├───────┤ ░  │ZZ(0)          ░ ├───────┤ ░  │ZZ(0)          ░ ├───────┤ ░ »
q_1: ┤ Rx(0) ├─░──■───────■───────░─┤ Ry(0) ├─░──■───────■───────░─┤ Rx(0) ├─░─»
     ├───────┤ ░          │ZZ(0)  ░ ├───────┤ ░          │ZZ(0)  ░ ├───────┤ ░ »
q_2: ┤ Rx(0) ├─░──────────■───────░─┤ Ry(0) ├─░──────────■───────░─┤ Rx(0) ├─░─»
     └───────┘ ░                  ░ └───────┘ ░                  ░ └───────┘ ░ »
«                      ░ ┌───────┐
«q_0: ─■───────────────░─┤ Ry(0) ├
«      │ZZ(0)          ░ ├───────┤
«q_1: ─■───────■───────░─┤ Ry(0) ├
«              │ZZ(0)  ░ ├───────┤
«q_2: ─────────■───────░─┤ Ry(0) ├
«                      ░ └───────┘
0.25 * IZZ
+ 0.25 * ZZI
+ 1.0 * IIX
+ 1.0 * IXI
+ 1.0 * XII
Sz_0
IIZ
Sx_0
IIX
Sy

/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_5742/2988717382.py:36: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  instance = QuantumInstance(backend=backend,shots=shots)
/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_5742/2988717382.py:43: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_op.PauliOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  obs['Sz_'+str(i)]      = PauliOp(generate_pauli([],[i],spins),1.0)
/var/folders/1k/wfwgsdr57xz4qwc3p5r0l92h0000gq/T/ipykernel_5742/2988717382.py:44: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_op.PauliOp`` is deprecated as of qiskit-terra 

In [7]:
### Initialize the algorithm

# Choose a specific set of parameters
initial_point = None

# Choose the gradient optimizer: 'sgd', 'adam'
opt  = 'sgd'
# Choose how to estimate the gradient on hardware: 'param_shift', 'spsa'
grad = 'param_shift'
# Choose which type of cost function use: 'global', 'local'
cost = 'local'


algo = pVQD(hamiltonian   = H,
			ansatz        = hweff_ansatz,
			ansatz_reps   = depth,
			parameters    = ex_params,
			initial_shift = shift,
			instance      = instance,
			shots         = shots)

algo.run(ths,dt,n_steps, 
	     obs_dict      = obs,
	     filename      = 'data/trial_results.dat',
	     max_iter      = 50,
	     opt           = opt,
	     cost_fun      = cost,
	     grad          = grad,
	     initial_point = initial_point)

Running the algorithm


Time slice: 1
Shift before optimizing this step: [0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01]
Initial parameters: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Shift optimizing step: 1
Overlap [9.97708333e-01 3.07944222e-04]
Gradient [ 0.01525     0.01310417  0.01266667  0.00025     0.         -0.00554167
 -0.00247917 -0.00125    -0.000125   -0.00025     0.01291667  0.01539583
  0.0114375  -0.0005625  -0.00039583 -0.00304167 -0.00254167 -0.004375  ]
Shift optimizing step: 2
Overlap [9.99208333e-01 1.81404986e-04]
Gradient [ 0.00797917  0.00797917  0.01070833 -0.00035417 -0.00033333 -0.00220833
 -0.00252083 -0.00183333 -0.0005625  -0.00047917  0.00885417  0.00483333
  0.0095625  -0.00045833 -0.00035417 -0.0025625  -0.00235417 -0.00095833]
Shift optimizing step: 3
Overlap [9.99583333e-01 1.31679192e-04]
Gradient [ 4.97916667e-03  4.95833333e-03  6.02083333e-03 -1.25000000e-04
  4.16666667e-05 -2.20833333e

Overlap [1. 0.]
Gradient [-2.29166667e-04  5.41666667e-04  6.04166667e-04  5.83333333e-04
  2.91666667e-04  5.62500000e-04 -4.16666667e-05  3.95833333e-04
  6.66666667e-04 -1.41666667e-03  8.12500000e-04  1.04166667e-04
  6.25000000e-04 -1.52083333e-03 -1.50000000e-03 -1.83333333e-03
 -1.70833333e-03 -4.58333333e-04]

---------------------------------- 

Shift after optimizing: [ 0.04727083  0.04679167  0.05410417  0.010875    0.00883333 -0.00254167
 -0.0125      0.000125    0.0096875   0.00622917  0.0520625   0.0505625
  0.04233333  0.0059375   0.00520833 -0.00260417  0.007625   -0.0025625 ]
New parameters: [ 0.2374375   0.23635417  0.2689375   0.051625    0.0426875  -0.00929167
 -0.0574375  -0.00091667  0.043375    0.0379375   0.25427083  0.24972917
  0.21014583  0.03510417  0.03414583 -0.0034375   0.040375   -0.01120833]
New overlap:  0.9999999999999999


Time slice: 6
Shift before optimizing this step: [ 0.04727083  0.04679167  0.05410417  0.010875    0.00883333 -0.00254167
 -0.012

Overlap [9.99958333e-01 4.16640624e-05]
Gradient [-3.95833333e-04  1.20833333e-03 -1.25000000e-04  1.12500000e-03
  7.91666667e-04 -8.75000000e-04  5.62500000e-04  2.29166667e-04
 -7.08333333e-04 -3.12500000e-04  7.70833333e-04 -1.22916667e-03
 -8.33333333e-05  1.81250000e-03 -5.41666667e-04  4.79166667e-04
  2.68750000e-03  3.12500000e-04]
Shift optimizing step: 2
Overlap [1. 0.]
Gradient [-8.33333333e-05 -1.00000000e-03  1.18750000e-03 -5.83333333e-04
  1.56250000e-03  1.72916667e-03  3.54166667e-04  1.39583333e-03
  8.75000000e-04  1.85416667e-03  4.16666667e-05  4.16666667e-05
  2.08333333e-05 -7.29166667e-04 -2.08333333e-04 -7.70833333e-04
  2.70833333e-04  6.25000000e-04]

---------------------------------- 

Shift after optimizing: [ 0.0405625   0.04847917  0.058375    0.01308333  0.008875   -0.00175
 -0.00754167 -0.00404167  0.0110625   0.00529167  0.0575      0.05216667
  0.04        0.00422917  0.0073125  -0.00475     0.00979167 -0.00060417]
New parameters: [ 0.5131875   0.53

Overlap [1. 0.]
Gradient [ 0.00039583 -0.00066667 -0.000375   -0.003       0.00039583  0.00108333
  0.00058333 -0.00047917 -0.00535417  0.00072917  0.00175     0.00129167
  0.00202083 -0.00279167 -0.00635417 -0.00025    -0.0015625   0.00085417]

---------------------------------- 

Shift after optimizing: [ 0.03952083  0.04972917  0.05733333  0.01372917  0.01035417 -0.0034375
  0.0004375  -0.00379167  0.00429167  0.01325     0.06227083  0.053375
  0.04372917  0.00016667 -0.004875   -0.00422917  0.00958333 -0.00220833]
New parameters: [ 0.75429167  0.82095833  0.9564375   0.2216875   0.15977083 -0.04554167
 -0.1325625  -0.03266667  0.15879167  0.13472917  0.95585417  0.87622917
  0.69597917  0.07566667  0.088875   -0.05354167  0.14591667 -0.02910417]
New overlap:  0.9999999999999999


Time slice: 18
Shift before optimizing this step: [ 0.03952083  0.04972917  0.05733333  0.01372917  0.01035417 -0.0034375
  0.0004375  -0.00379167  0.00429167  0.01325     0.06227083  0.053375
  0.04372917

Overlap [9.99958333e-01 4.16640624e-05]
Gradient [-0.00202083 -0.00058333  0.001375   -0.00597917 -0.006375   -0.000875
 -0.001      -0.00079167  0.0035     -0.00339583  0.00391667  0.001
  0.00197917 -0.00077083 -0.00227083  0.0015      0.0016875   0.00185417]
Shift optimizing step: 2
Overlap [1. 0.]
Gradient [-0.0008125   0.0001875  -0.0010625   0.00225     0.0010625  -0.00135417
  0.000125    0.0003125  -0.00083333  0.00175    -0.00116667 -0.00327083
 -0.00147917 -0.00139583  0.0053125  -0.00139583 -0.00225    -0.00022917]

---------------------------------- 

Shift after optimizing: [ 0.03889583  0.05408333  0.0589375   0.01147917 -0.00285417 -0.00883333
  0.007      -0.0025625  -0.00158333  0.01229167  0.06547917  0.04795833
  0.04272917  0.00122917  0.00339583 -0.00133333  0.00883333  0.0039375 ]
New parameters: [ 0.96058333  1.0848125   1.24697917  0.29514583  0.1795     -0.07429167
 -0.10389583 -0.04914583  0.1556875   0.19479167  1.27216667  1.134625
  0.91579167  0.08560417  

Overlap [1. 0.]
Gradient [-1.20833333e-03  2.70833333e-04 -1.77083333e-03 -5.83333333e-04
  9.58333333e-04 -3.12500000e-04  8.95833333e-04 -6.25000000e-05
  3.75000000e-03  2.66666667e-03 -1.04166667e-04  2.06250000e-03
 -7.08333333e-04  5.20833333e-04 -1.77083333e-03 -1.25000000e-03
 -1.83333333e-03  2.50000000e-04]

---------------------------------- 

Shift after optimizing: [ 0.0396875   0.05179167  0.05854167  0.01054167 -0.00602083 -0.00552083
  0.01141667 -0.00335417 -0.00385417  0.01847917  0.06935417  0.051625
  0.042875    0.00245833  0.00164583 -0.00495833 -0.00022917  0.00129167]
New parameters: [ 1.19877083e+00  1.39870833e+00  1.60389583e+00  3.67083333e-01
  1.52416667e-01 -1.09812500e-01 -4.80208333e-02 -6.73541667e-02
  1.34895833e-01  2.81541667e-01  1.68314583e+00  1.43260417e+00
  1.16864583e+00  9.85833333e-02  1.04791667e-01 -7.66250000e-02
  2.26625000e-01 -4.58333333e-04]
New overlap:  0.9999999999999999


Time slice: 29
Shift before optimizing this step: [ 0.03

Overlap [9.99875000e-01 9.31590169e-05]
Gradient [-1.08333333e-03 -1.70833333e-03 -2.08333333e-04 -2.29166667e-04
 -1.62500000e-03  3.95833333e-04 -2.79166667e-03 -5.62500000e-04
  2.72916667e-03  8.33333333e-05  6.45833333e-04 -3.12500000e-04
 -1.06250000e-03  2.29166667e-03  6.04166667e-04 -1.43750000e-03
 -1.66666667e-03 -8.75000000e-04]
Shift optimizing step: 3
Overlap [1. 0.]
Gradient [-1.50000000e-03  3.29166667e-03  1.20833333e-03 -7.27083333e-03
 -1.02083333e-03  3.54166667e-04  1.35416667e-03  2.08333333e-04
  1.95833333e-03 -2.29166667e-03  2.08333333e-05  1.27083333e-03
 -2.95833333e-03  2.04166667e-03 -2.27083333e-03 -2.60416667e-03
 -2.50000000e-03  2.08333333e-04]

---------------------------------- 

Shift after optimizing: [ 0.02620833  0.05195833  0.0571875   0.0073125  -0.009625   -0.01020833
  0.00477083 -0.00777083 -0.0129375   0.01572917  0.06804167  0.05433333
  0.0404375   0.01114583  0.00166667 -0.01216667 -0.01647917 -0.00370833]
New parameters: [ 1.35833333e+0

Overlap [9.99833333e-01 1.17836398e-04]
Gradient [-0.00016667 -0.00091667 -0.00125     0.00222917 -0.00225     0.00245833
 -0.00104167 -0.00241667 -0.00370833 -0.00460417  0.00025     0.0005625
  0.00127083  0.00377083  0.00095833 -0.00060417  0.00114583  0.0003125 ]
Shift optimizing step: 2
Overlap [9.99916667e-01 8.33281248e-05]
Gradient [-1.39583333e-03  7.70833333e-04 -2.50000000e-04  3.10416667e-03
  2.27083333e-03  7.29166667e-04 -6.45833333e-04  9.79166667e-04
 -3.66666667e-03 -4.16666667e-05 -2.22916667e-03  5.20833333e-04
 -6.87500000e-04 -1.66666667e-04  5.62500000e-04 -5.14583333e-03
 -2.91666667e-04  1.04166667e-03]
Shift optimizing step: 3
Overlap [9.99875000e-01 9.31590169e-05]
Gradient [ 2.50000000e-03  1.58333333e-03  2.50000000e-04  4.56250000e-03
  8.95833333e-04 -1.37500000e-03 -1.70833333e-03  1.54166667e-03
 -2.91666667e-04  8.33333333e-05  1.29166667e-03  1.12500000e-03
 -9.58333333e-04  4.41666667e-03  3.64583333e-03  3.06250000e-03
  1.72916667e-03 -8.33333333e-